In [9]:
import json
import hashlib
from flask import Flask, jsonify
import datetime
import os

In [10]:
class LiteCoinChain:
    def __init__(self):
        self.chain = []
        self.addCoin(proof = 1, previousHash = "0")
        
    def addCoin(self, proof, previousHash):
        coin = {
            "index": len(self.chain) + 1,
            "timestamp": str(datetime.datetime.now()),
            "proof": proof,
            "previoushash": previousHash
        }
        self.chain.append(coin)
        return coin
    
    def getLastCoin(self):
        return self.chain[-1]
    
    def proofOfWork(self, previousProof):
        new_proof = 1
        solved = False
        while solved is False:
            hashProcess = hashlib.sha256(str(new_proof **2 - previousProof ** 2).encode()).hexdigest()
            if hashProcess[:4] == "0000":
                solved = True
            else:
                new_proof += 1
        return new_proof
    
    def hashCoin(self, coin):
        encodedHash = json.dumps(coin).encode()
        return hashlib.sha256(encodedHash).hexdigest()
    
    def isChainVaild(self, lightChain):
        previousCoin = lightChain[0]
        current_index = 1
        
        while current_index < len(lightChain):
            currentCoin = lightChain[current_index]            
            if currentPrev["previoushash"] != hashCoin(previousCoin):
                return  False
            currentProof = currentCoin["proof"]
            prevProof= previousCoin["proof"]
            hashCheck = hashlib.sha256(str(currentProof **2 - prevProof ** 2).encode()).hexdigest()
            if hashCheck[:4] != "0000":
                return False
        return True

In [11]:
LTE = LiteCoinChain()

In [12]:
app = Flask(__name__)

In [13]:
@app.route("/mine", methods = ["GET"])
def mine():
    prevCoin = LTE.getLastCoin()
    prevProof = prevCoin["proof"]
    proof = LTE.proofOfWork(prevProof)
    prev_hash = LTE.hashCoin(prevCoin)
    coin = LTE.addCoin(proof, prev_hash)
    response = {
        "success":"You have mined a coin :)",
        "index": coin["index"],
        "proof": coin["proof"],
        "prev": coin["previoushash"],
        "timestamp": coin["timestamp"] 
    }
    return jsonify(response), 200

In [14]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        "chain": LTE.chain,
        "len": len(LTE.chain)
    }
    return jsonify(response)

In [15]:
@app.route("/valid", methods = ["GET"])
def valid():
    response = {}
    if LTE.isChainVaild(LTE.chain) is True:
        response["validResponse"] = "This chain is valid"
    else:
        response["validResponse"] = "This chain is invalid"
    return jsonify(response)

In [16]:
if __name__=="__main__":
    app.run(host=os.getenv('IP', '0.0.0.0'), 
            port=int(os.getenv('PORT', 4444)))

 * Running on http://0.0.0.0:4444/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Dec/2021 10:49:01] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 10:49:05] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 10:49:06] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 10:49:07] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 10:49:10] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 10:49:42] "GET /vaild HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2021 10:50:12] "GET /vaild HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2021 10:50:43] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 10:50:48] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 10:50:50] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 10:50:57] "GET /chain HTTP/1.1" 200 -
